# Evaluate on a Target

An Ask Wiki app has been created that uses the Wikipedia API to answer questions leveraging information available in Wikipedia articles.

In this exercise, you will assess the relevance of the chatbot's responses given a query.

## Add environment variables to the .env file

In the root of the **Evaluation and Data Generation Workshop** folder is an `.env` file. Within the `.env` file, fill in the values for the environment variables. You can locate the values for each environment variable in the following locations of the [Azure AI Foundry](https://ai.azure.com) portal:

- `AZURE_SUBSCRIPTION_ID` - On the **Overview** page of your project within **Project details**.
- `AZURE_AI_PROJECT_NAME` - At the top of the **Overview** page for your project.
- `AZURE_OPENAI_RESOURCE_GROUP` - On the **Overview** page of the **Management Center** within **Project properties**.
- `AZURE_OPENAI_SERVICE` - On the **Overview** page of your project in the **Included capabilities** tab for **Azure OpenAI Service**.
- `AZURE_OPENAI_API_VERSION` - On the [API version lifecycle](https://learn.microsoft.com/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release) webpage within the **Latest GA API release** section.
- `AZURE_OPENAI_ENDPOINT` - On the **Details** tab of your model deployment within **Endpoint** (i.e. **Target URI**)
- `AZURE_OPENAI_DEPLOYMENT_NAME` -  On the **Details** tab of your model deployment within **Deployment info**.

# Sign in to Azure

As a security best practice, we'll use [keyless authentication](https://learn.microsoft.com/azure/developer/ai/keyless-connections?tabs=csharp%2Cazure-cli) to authenticate to Azure OpenAI with Microsoft Entra ID. Before you can do so, you'll first need to install the **Azure CLI** per the [installation instructions](https://learn.microsoft.com/cli/azure/install-azure-cli) for your operating system.

Next, open a terminal and run `az login` to sign in to your Azure account.

## Import and Test Ask Wiki

Let's test a query with Ask Wiki to validate that your environment variables are properly configured. We'll begin by importing the `ask_wiki` function from `askwiki`. The `ask_wiki` function generates a response from the app. Once imported, we'll pass in a query to view the response and context generated.

In [ ]:
%pip install bs4

from askwiki import ask_wiki

ask_wiki(query="What is the capital of India?")

## Install the package

The `evaluate` function for evaluating on a target, and the evaluator class for assessing relevance is in the Azure AI Evaluation SDK. We'll begin by installing the package.

In [ ]:
%pip install azure-ai-evaluation

## Access the environment variables.

We'll import `os` and `load_dotenv` so that you can access the environment variables.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

## Import packages

We'll now import the `evaluate` function and `RelevanceEvaluator` class. We'll also import some additional libraries to help with accessing our data and formatting the results.


In [ ]:
from azure.ai.evaluation import evaluate, RelevanceEvaluator
import pandas as pd
from pprint import pprint

## Setup keyless authentication

Rather than hardcode your **key**, we'll use a keyless connection with Azure OpenAI.

In [ ]:
import azure.identity

credential = azure.identity.DefaultAzureCredential()
token_provider = azure.identity.get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

token = token_provider()

## Configure the model_config

The `model_config` is necessary as it's a required parameter when creating an instance of the evaluator class. Let's configure the `model_config` with the following:

- Azure deployment name
- Azure OpenAI endpoint
- OpenAI API version
- Azure OpenAI API Key

In [ ]:
model_config = {
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

## Create an instance of the evaluator

Let's now create an instance of the `RelevanceEvaluator`.

In [ ]:
relevance_eval = RelevanceEvaluator(model_config)

## Create the call to evaluate on a target

We can run an evaluation on a target with the `evaluate` function and list our evaluator. Let's assign this function call to the `results` variable. We'll later use this variable to format and print the results.

In [ ]:
results = evaluate(
    data="data.jsonl",
    target=ask_wiki,
    evaluators={
        "relevance": relevance_eval,
    }
)

## Print the results with Pretty Print

Now that we've run the evaluation, let's print the results using `pretty print`, which displays data in a structured and visually appealing way, making it easier to read and understand.

In [ ]:
pprint(results)

## Print the results as table

We can also print the results as a table using `Pandas`.

In [ ]:
pd.DataFrame(results["rows"])

## Delete resources

If you've finished exploring Azure AI Services, delete the Azure resource that you created during the workshop.

**Note**: You may be prompted to delete your deployed model(s) before deleting the resource group.